In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import gc
import PIL
import tensorflow as tf
from tensorflow import keras 
from keras.callbacks import EarlyStopping


In [ ]:
class ModelTrain:
    es=EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=5,restore_best_weights=True,start_from_epoch=3)
    def __init__(self,main_dir):
        main_dir=main_dir
        
    def imgDataGen(self):
        train_datagen = keras.preprocessing.image.ImageDataGenerator(
            rescale=1./255,
            rotation_range=45,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.4,
            horizontal_flip=True,
            fill_mode='nearest'
        )
        test_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
        valid_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
        return train_datagen,test_datagen,valid_datagen
    
    def flowImgData(self,main_dir,datagen):
        generator = datagen.flow_from_directory(
            main_dir,
            target_size=(244, 244),
            batch_size=32,
            class_mode='sparse'
        )
        print("Generator Class Indices : ",generator.class_indices)
        print("Number of Classes : ",len(generator.class_indices))
        return generator
    
    def model(self,num_classes):
        model = keras.models.Sequential([
            keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(244, 244, 3)),
            keras.layers.MaxPooling2D(2, 2),
            keras.layers.Conv2D(64, (3,3), activation='relu'),
            keras.layers.MaxPooling2D(2,2),
            keras.layers.Conv2D(128, (3,3), activation='relu'),
            keras.layers.MaxPooling2D(2,2),
            keras.layers.Conv2D(128, (3,3), activation='relu'),
            keras.layers.MaxPooling2D(2,2),
            keras.layers.Flatten(),
            keras.layers.Dropout(0.3),
            keras.layers.Dense(256, activation='relu'),
            keras.layers.Dense(num_classes, activation='softmax')
        ])
        model.compile(
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
            optimizer='adam',
            metrics=['accuracy']
        )
        return model
    
    def trainModel(self,train_generator,valid_generator,model,save_path=""):
        history = model.fit(
            train_generator,
            epochs=100,
            validation_data=valid_generator,
            verbose=1,
            callbacks=[self.es]
        )
        if(save_path!=""):
            model.save(save_path)
        return history,model
    
    def plotHistory(self,history,crop):
        acc = history.history['accuracy']
        val_acc = history.history['val_accuracy']
        loss = history.history['loss']
        val_loss = history.history['val_loss']
        epochs = range(len(acc))
        plt.plot(epochs, acc, 'r', label='Training accuracy')
        plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
        plt.title('Training and validation accuracy')
        plt.legend(loc=0)
        plt.figure()
        plt.show()
        plt.savefig(f"../Dataset_l/crop_specific_models/{crop}_acc_vs_val_acc.png")
        plt.plot(epochs, loss, 'r', label='Training Loss')
        plt.plot(epochs, val_loss, 'b', label='Validation Loss')
        plt.title('Training and validation loss')
        plt.legend(loc=0)
        plt.figure()
        plt.show()
        plt.savefig(f"../Dataset_l/crop_specific_models/{crop}_loss_vs_val_loss.png")
    
    def predict(self,model,img_path):
        img = keras.preprocessing.image.load_img(
            img_path, target_size=(244, 244)
        )
        img_array = keras.preprocessing.image.img_to_array(img)
        img_array = tf.expand_dims(img_array, 0)
        predictions = model.predict(img_array)
        score = tf.nn.softmax(predictions[0])
        confidence=np.argmax(score)
        return predictions,confidence
    
    def testAccuracy(self,test_generator,model):
        test_loss, test_acc = model.evaluate(test_generator, verbose=2)
        print("Test Accuracy : ",test_acc)
        return test_acc
    
    def predictAll(self,model,dir):
        predictionArray=[]
        confidenceArray=[]
        for i in os.listdir(dir):
            img_path=os.path.join(dir,i)
            pred,confidence=self.predict(model,img_path)
            predictionArray.append(pred)
            confidenceArray.append(confidence)
            print("Prediction : ",pred)
            print("Confidence : ",confidence)
            print("Image Path : ",img_path)
            print("Class : ",i)
            print("--------------------------------------------------------")
        predictionArray=np.array(predictionArray)
        confidenceArray=np.array(confidenceArray)
        return predictionArray,confidenceArray
    
    

In [ ]:
data_dir="../Dataset_l/individual_datasets"
for i in sorted(os.listdir(data_dir)):
    print("---------------------------------------------")
    print("Crop Name : ",i)
    print("---------------------------------------------")
    main_dir=os.path.join(data_dir,i)
    print("Directory : ",main_dir)
    print("---------------------------------------------")
    modelTrain=ModelTrain(main_dir)
    
    train_datagen,test_datagen,valid_datagen=modelTrain.imgDataGen()

    train_generator=modelTrain.flowImgData(os.path.join(main_dir,"train"),train_datagen)
    test_generator=modelTrain.flowImgData(os.path.join(main_dir,"test"),test_datagen)
    valid_generator=modelTrain.flowImgData(os.path.join(main_dir,"val"),valid_datagen)
    
    num_classes=len(train_generator.class_indices)
    
    model=modelTrain.model(num_classes)
    history,model=modelTrain.trainModel(train_generator,valid_generator,model,save_path=os.path.join("../Dataset_l/crop_specific_models",f"{i}_model.pb"))
    modelTrain.plotHistory(history,i)
    modelTrain.testAccuracy(test_generator,model)
    gc.collect()
    print("--------------------------------------------------------")
    
